## Fitting a diagonal covariance Gaussian mixture model to text data

In a previous assignment, we explored k-means clustering for a high-dimensional Wikipedia dataset. We can also model this data with a mixture of Gaussians, though with increasing dimension we run into two important issues associated with using a full covariance matrix for each component.
 * Computational cost becomes prohibitive in high dimensions: score calculations have complexity cubic in the number of dimensions M if the Gaussian has a full covariance matrix.
 * A model with many parameters require more data: observe that a full covariance matrix for an M-dimensional Gaussian will have M(M+1)/2 parameters to fit. With the number of parameters growing roughly as the square of the dimension, it may quickly become impossible to find a sufficient amount of data to make good inferences.

Both of these issues are avoided if we require the covariance matrix of each component to be diagonal, as then it has only M parameters to fit and the score computation decomposes into M univariate score calculations. Recall from the lecture that the M-step for the full covariance is:

\begin{align*}
\hat{\Sigma}_k &= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_i-\hat{\mu}_k)(x_i - \hat{\mu}_k)^T
\end{align*}

Note that this is a square matrix with M rows and M columns, and the above equation implies that the (v, w) element is computed by

\begin{align*}
\hat{\Sigma}_{k, v, w} &= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_{iv}-\hat{\mu}_{kv})(x_{iw} - \hat{\mu}_{kw})
\end{align*}

When we assume that this is a diagonal matrix, then non-diagonal elements are assumed to be zero and we only need to compute each of the M elements along the diagonal independently using the following equation. 

\begin{align*}
\hat{\sigma}^2_{k, v} &= \hat{\Sigma}_{k, v, v}  \\
&= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_{iv}-\hat{\mu}_{kv})^2
\end{align*}

In this section, we will use an EM implementation to fit a Gaussian mixture model with **diagonal** covariances to a subset of the Wikipedia dataset. The implementation uses the above equation to compute each variance term. 

We'll begin by importing the dataset and coming up with a useful representation for each article. After running our algorithm on the data, we will explore the output to see whether we can give a meaningful interpretation to the fitted parameters in our model.

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import spdiags
from scipy.stats import multivariate_normal
from copy import deepcopy
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

In [3]:
!ls

(OPTIONAL) A worked-out example for EM _ Coursera.pdf
4_map_index_to_word.gl
4_map_index_to_word.json
4_tf_idf.npz
CLU04-NB01.ipynb
CLU04-NB02.ipynb
Clustering_week4_assignment_1.ipynb
DCphrd4eEemzYQ60aJAwVA_fc87f6033fa34a2ab644e47ef86539cc_em_utilities.py.zip
EXa8L-I8EemELQpo9cj5Ig_47f889d69bf74016adb595f66fad3bec_CLU04-NB02.ipynb.zip
EXiRLuI8Eemm5A4ynZyB2A_02d835d967234a7abc91cf61451f8ff0_CLU04-NB01.ipynb.zip
_167fc84b78dc145609e919983b475aaa_people_wiki.csv.zip
_8022d9156b93197ae8878c79eba9767b_4_map_index_to_word.gl.zip
_8022d9156b93197ae8878c79eba9767b_4_map_index_to_word.json.zip
_8a352ed8c119f98f079305de3bd46e26_images.zip
_af95de312b911daa1d7142fd60ae12e1_4_tf_idf.npz.zip
clustering_week4_assignment_2.ipynb
em_utilities.py
images
mixmodel-EM-annotated.pdf
people_wiki.csv


In [4]:
wiki = pd.read_csv("people_wiki.csv").head(5000)

As in the previous assignment, we extract the TF-IDF vector of each document.

For your convenience, we extracted the TF-IDF vectors from the dataset. The vectors are packaged in a sparse matrix, where the i-th row gives the TF-IDF vectors for the i-th document. Each column corresponds to a unique word appearing in the dataset.

To load in the TF-IDF vectors, run

In [5]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

In [6]:
tf_idf = load_sparse_csr('4_tf_idf.npz')  # NOT people_wiki_tf_idf.npz

In [8]:
import json

with open('4_map_index_to_word.json', 'r') as filehandle:
    map_index_to_word = json.load(filehandle)

In [9]:
1/0.40

2.5

In [12]:
1/np.sqrt(2)

0.7071067811865475

As in the previous assignment, we will normalize each document's TF-IDF vector to be a unit vector.

In [13]:
tf_idf = normalize(tf_idf)

(Optional) Extracting TF-IDF vectors yourself. We provide the pre-computed TF-IDF vectors to minimize potential compatibility issues. You are free to experiment with other tools to compute the TF-IDF vectors yourself. A good place to start is sklearn.TfidfVectorizer. Note. Due to variations in tokenization and other factors, your TF-IDF vectors may differ from the ones we provide. For the purpose the assessment, we ask you to use the vectors from 4_tf_idf.npz.

## EM in high dimensions

EM for high-dimensional data requires some special treatment:
 * E step and M step must be vectorized as much as possible, as explicit loops are dreadfully slow in Python.
 * All operations must be cast in terms of sparse matrix operations, to take advantage of computational savings enabled by sparsity of data.
 * Initially, some words may be entirely absent from a cluster, causing the M step to produce zero mean and variance for those words.  This means any data point with one of those words will have 0 probability of being assigned to that cluster since the cluster allows for no variability (0 variance) around that count being 0 (0 mean). Since there is a small chance for those words to later appear in the cluster, we instead assign a small positive variance (~1e-10). Doing so also prevents numerical overflow.
 
We provide the complete implementation for you in the file `em_utilities.py`. For those who are interested, you can read through the code to see how the sparse matrix implementation differs from the previous assignment. 

You are expected to answer some quiz questions using the results of clustering.

Due to complexity in implementation, we provide the complete implementation here. You are expected to answer some quiz questions using the results of clustering.

### Log probability function for diagonal covariance Gaussian.

In [15]:
def diag(array):
    n = len(array)
    return spdiags(array, 0, n, n)

def logpdf_diagonal_gaussian(x, mean, cov):
    '''
    Compute logpdf of a multivariate Gaussian distribution with diagonal covariance at a given point x.
    A multivariate Gaussian distribution with a diagonal covariance is equivalent
    to a collection of independent Gaussian random variables.

    x should be a sparse matrix. The logpdf will be computed for each row of x.
    mean and cov should be given as 1D numpy arrays
    mean[i] : mean of i-th variable
    cov[i] : variance of i-th variable'''

    n = x.shape[0]
    dim = x.shape[1]
    assert(dim == len(mean) and dim == len(cov))

    # multiply each i-th column of x by (1/(2*sigma_i)), where sigma_i is sqrt of variance of i-th variable.
    scaled_x = x.dot( diag(1./(2*np.sqrt(cov))) )
    # multiply each i-th entry of mean by (1/(2*sigma_i))
    scaled_mean = mean/(2*np.sqrt(cov))

    # sum of pairwise squared Eulidean distances gives SUM[(x_i - mean_i)^2/(2*sigma_i^2)]
    return -np.sum(np.log(np.sqrt(2*np.pi*cov))) - pairwise_distances(scaled_x, [scaled_mean], 'euclidean').flatten()**2

### EM algorithm for sparse data.

In [22]:
def log_sum_exp(x, axis):
    '''Compute the log of a sum of exponentials'''
    x_max = np.max(x, axis=axis)
    if axis == 1:
        return x_max + np.log( np.sum(np.exp(x-x_max[:,np.newaxis]), axis=1) )
    else:
        return x_max + np.log( np.sum(np.exp(x-x_max), axis=0) )

def EM_for_high_dimension(data, means, covs, weights, cov_smoothing=1e-5, maxiter=int(1e3), thresh=1e-4, verbose=False):
    # cov_smoothing: specifies the default variance assigned to absent features in a cluster.
    #                If we were to assign zero variances to absent features, we would be overconfient,
    #                as we hastily conclude that those featurese would NEVER appear in the cluster.
    #                We'd like to leave a little bit of possibility for absent features to show up later.
    n = data.shape[0]
    dim = data.shape[1]
    mu = deepcopy(means)
    Sigma = deepcopy(covs)
    K = len(mu)
    weights = np.array(weights)

    ll = None
    ll_trace = []

    for i in range(maxiter):
        # E-step: compute responsibilities
        logresp = np.zeros((n,K))
        for k in range(K):
            logresp[:,k] = np.log(weights[k]) + logpdf_diagonal_gaussian(data, mu[k], Sigma[k])
        ll_new = np.sum(log_sum_exp(logresp, axis=1))
        if verbose:
            print(ll_new)
        logresp -= np.vstack(log_sum_exp(logresp, axis=1))
        resp = np.exp(logresp)
        counts = np.sum(resp, axis=0)

        # M-step: update weights, means, covariances
        weights = counts / np.sum(counts)
        for k in range(K):
            mu[k] = (diag(resp[:,k]).dot(data)).sum(axis=0)/counts[k]
            mu[k] = mu[k].A1

            Sigma[k] = diag(resp[:,k]).dot( data.multiply(data)-2*data.dot(diag(mu[k])) ).sum(axis=0) \
                       + (mu[k]**2)*counts[k]
            Sigma[k] = Sigma[k].A1 / counts[k] + cov_smoothing*np.ones(dim)

        # check for convergence in log-likelihood
        ll_trace.append(ll_new)
        if ll is not None and (ll_new-ll) < thresh and ll_new > -np.inf:
            ll = ll_new
            break
        else:
            ll = ll_new

    out = {'weights':weights,'means':mu,'covs':Sigma,'loglik':ll_trace,'resp':resp}

    return out

**Initializing mean parameters using k-means**

Recall from the lectures that EM for Gaussian mixtures is very sensitive to the choice of initial means. With a bad initial set of means, EM may produce clusters that span a large area and are mostly overlapping. To eliminate such bad outcomes, we first produce a suitable set of initial means by using the cluster centers from running k-means.  That is, we first run k-means and then take the final set of means from the converged solution as the initial means in our EM algorithm.

In [17]:
from sklearn.cluster import KMeans

np.random.seed(5)
num_clusters = 25

# Use scikit-learn's k-means to simplify workflow
kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=-1)
kmeans_model.fit(tf_idf)
centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_

means = [centroid for centroid in centroids]

**Initializing cluster weights.** We will initialize each cluster weight to be the proportion of documents assigned to that cluster by k-means above.

In [19]:
num_docs = tf_idf.shape[0]
weights = []
for i in range(num_clusters):
    # Compute the number of data points assigned to cluster i:
    num_assigned = sum(cluster_assignment == i) # YOUR CODE HERE
    w = float(num_assigned) / num_docs
    weights.append(w)

**Initializing covariances**

To initialize our covariance parameters, we compute $\hat{\sigma}_{k, j}^2 = \sum_{i=1}^{N}(x_{i,j} - \hat{\mu}_{k, j})^2$ for each feature $j$.  For features with really tiny variances, we assign 1e-8 instead to prevent numerical instability. We do this computation in a vectorized fashion in the following code block.

In [20]:
covs = []
for i in range(num_clusters):
    member_rows = tf_idf[cluster_assignment==i]
    cov = (member_rows.multiply(member_rows) - 2*member_rows.dot(diag(means[i]))).sum(axis=0).A1 / member_rows.shape[0] \
          + means[i]**2
    cov[cov < 1e-8] = 1e-8
    covs.append(cov)

**Running EM**

Now that we have initialized all of our parameters, run EM.

In [23]:
out = EM_for_high_dimension(tf_idf, means, covs, weights, cov_smoothing=1e-10)

In [24]:
out['loglik']

[3855847476.7012835, 4844053202.46348, 4844053202.46348]

## Interpret clustering results

In contrast to k-means, EM is able to explicitly model clusters of varying sizes and proportions. The relative magnitude of variances in the word dimensions tell us much about the nature of the clusters.

Write yourself a cluster visualizer as follows.  Examining each cluster's mean vector, list the 5 words with the largest mean values (5 most common words in the cluster). For each word, also include the associated variance parameter (diagonal element of the covariance matrix). 

A sample output may be:
```
==========================================================
Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
football    1.08e-01    8.64e-03
season      5.80e-02    2.93e-03
club        4.48e-02    1.99e-03
league      3.94e-02    1.08e-03
played      3.83e-02    8.45e-04
...
```

In [34]:
tf_idf.shape

(5000, 100282)

In [38]:
len(out['means'][0])

100282

In [39]:
print(out['means'][0])

[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.01954039e-04
 1.04447276e-04 9.87192763e-05]


In [42]:
np.argsort(means[0])  ##[::-1]

array([     0,  62037,  62036, ..., 100039,  99951,  99830])

In [43]:
np.argsort(means[0])[::-1]

array([ 99830,  99951, 100039, ...,  62036,  62037,      0])

In [47]:
x = np.array([20,4,21,5,10])   ## returns the index of the sorted array
np.argsort(x)

array([1, 3, 4, 0, 2])

In [58]:
# Fill in the blanks
def visualize_EM_clusters(tf_idf, means, covs, map_index_to_word):
    print('')
    print('==========================================================')
    
    num_clusters = len(means)
    for c in range(num_clusters):
        print('Cluster {0:d}: Largest mean parameters in cluster '.format(c))
        print('\n{0: <12}{1: <12}{2: <12}'.format('Word', 'Mean', 'Variance'))
        
        # The k'th element of sorted_word_ids should be the index of the word 
        # that has the k'th-largest value in the cluster mean. Hint: Use np.argsort().
        sorted_word_ids = np.argsort(means[c])[::-1]  ## returns the index of the sorted array in descending order

        for i in sorted_word_ids[:5]:
            print('{0: <12}{1:<10.2e}{2:10.2e}'.format({v:k for k, v in map_index_to_word.items()}[i], 
                                                       means[c][i],
                                                       covs[c][i]))
        print('\n==========================================================')

In [59]:
dic_ = {'sundayswadsworth': 37318,
 'verplank': 46431,
 'damfunk': 23253,
 'anyer': 62437}
{v:k for k, v in dic_.items()}[46431]

'verplank'

In [60]:
map_index_to_word

{'sundayswadsworth': 37318,
 'verplank': 46431,
 'damfunk': 23253,
 'anyer': 62437,
 'sonja': 37922,
 'degussa': 33295,
 'heraldleader': 55678,
 'vinalop': 38962,
 'chameleons': 15539,
 'cfe': 19068,
 'spiders': 76384,
 'loanstopshop': 6262,
 'hanging': 85150,
 'woody': 94338,
 'suzana': 67596,
 'comically': 57840,
 'localized': 44264,
 'yougoslavia': 7189,
 'sprague': 16230,
 'gteborg': 82999,
 'chanthaburi': 17970,
 'igual': 52391,
 'bandmeyns': 10335,
 'unnecessarily': 65916,
 'storiesin': 84637,
 'canek': 27791,
 'petrak': 44943,
 'naturopathic': 42153,
 'bratislava': 86932,
 'heroesin': 23118,
 'theoryhe': 74282,
 'broward': 85834,
 'screaming': 85122,
 'petras': 70455,
 'receiving': 99209,
 'grueling': 74796,
 'vibrational': 53741,
 'kangema': 42859,
 'dcduring': 33588,
 'elgan': 57644,
 'wednesday': 93707,
 'millermost': 6231,
 'nielsenfunk': 24347,
 'crotch': 13410,
 'elgar': 70186,
 '598m': 20672,
 '196769': 68867,
 'shows': 99850,
 'mcelhinnycolleen': 2399,
 'bienniale': 5448

In [61]:
'''By EM'''
visualize_EM_clusters(tf_idf, out['means'], out['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
minister    7.57e-02    7.42e-03
election    5.89e-02    3.21e-03
party       5.89e-02    2.61e-03
liberal     2.93e-02    4.55e-03
elected     2.91e-02    8.95e-04

Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
film        1.76e-01    6.07e-03
films       5.50e-02    2.97e-03
festival    4.66e-02    3.60e-03
feature     3.69e-02    1.81e-03
directed    3.39e-02    2.22e-03

Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
art         1.26e-01    6.83e-03
museum      5.62e-02    7.27e-03
gallery     3.65e-02    3.40e-03
artist      3.61e-02    1.44e-03
design      3.20e-02    4.59e-03

Cluster 3: Largest mean parameters in cluster 

Word        Mean        Variance    
basketball  1.86e-01    7.78e-03
nba         1.01e-01    1.22e-02
points      6.25e-02    5.92e-03
coach       5.57e-02    5.91e-03
team        4.68e-02    1.30e

**Quiz Question**. Select all the topics that have a cluster in the model created above. [multiple choice]

- Baseball
- Basketball
- Soccer/Football
- Music
- Politics
- Law
- Finance

## Comparing to random initialization

Create variables for randomly initializing the EM algorithm. Complete the following code block.

In [62]:
tf_idf.shape

(5000, 100282)

In [63]:
np.random.uniform(1, 5, 10)

array([1.88797268, 4.48292922, 1.82687662, 4.67444363, 2.95364476,
       3.44697545, 4.06363143, 3.07367195, 2.18720201, 1.75088491])

In [64]:
np.random.seed(5) # See the note below to see why we set seed=5.
num_clusters = len(means)
num_docs, num_words = tf_idf.shape

random_means = []
random_covs = []
random_weights = []

for k in range(num_clusters):
    
    # Create a numpy array of length num_words with random normally distributed values.
    # Use the standard univariate normal distribution (mean 0, variance 1).
    # YOUR CODE HERE
    mean = np.random.normal(0, 1, num_words)
    
    # Create a numpy array of length num_words with random values uniformly distributed between 1 and 5.
    # YOUR CODE HERE
    cov = np.random.uniform(1, 5, num_words)

    # Initially give each cluster equal weight.
    # YOUR CODE HERE
    weight = 1./num_clusters
    
    random_means.append(mean)
    random_covs.append(cov)
    random_weights.append(weight)

In [70]:
len(random_weights)

25

In [72]:
np.shape(random_means)

(25, 100282)

In [76]:
#random_means

In [77]:
np.shape(random_covs)

(25, 100282)

**Quiz Question**: Try fitting EM with the random initial parameters you created above. (Use `cov_smoothing=1e-5`.) Store the result to `out_random_init`. What is the final loglikelihood that the algorithm converges to?

In [65]:
out_random_init = EM_for_high_dimension(tf_idf, means, covs, weights, cov_smoothing=1e-5)

In [66]:
out_random_init['loglik']

[3855847476.7012835, 2362779935.011491]

**Quiz Question:** Is the final loglikelihood larger or smaller than the final loglikelihood we obtained above when initializing EM with the results from running k-means?

In [67]:
out['loglik']

[3855847476.7012835, 4844053202.46348, 4844053202.46348]

**Quiz Question**: For the above model, `out_random_init`, use the `visualize_EM_clusters` method you created above. Are the clusters more or less interpretable than the ones found after initializing using k-means?

In [68]:
# YOUR CODE HERE. Use visualize_EM_clusters, which will require you to pass in tf_idf and map_index_to_word.
visualize_EM_clusters(tf_idf, out_random_init['means'], out_random_init['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
minister    7.47e-02    7.40e-03
election    5.85e-02    3.23e-03
party       5.85e-02    2.62e-03
elected     2.89e-02    9.13e-04
liberal     2.89e-02    4.50e-03

Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
film        1.70e-01    6.80e-03
films       5.33e-02    2.98e-03
festival    4.53e-02    3.55e-03
feature     3.58e-02    1.80e-03
directed    3.29e-02    2.20e-03

Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
art         1.16e-01    7.43e-03
museum      5.17e-02    6.86e-03
gallery     3.32e-02    3.21e-03
artist      3.32e-02    1.41e-03
design      3.20e-02    4.40e-03

Cluster 3: Largest mean parameters in cluster 

Word        Mean        Variance    
basketball  1.60e-01    1.07e-02
nba         8.66e-02    1.17e-02
points      5.42e-02    5.52e-03
coach       4.91e-02    5.39e-03
team        4.17e-02    1.35e

**Note**: Random initialization may sometimes produce a superior fit than k-means initialization. We do not claim that random initialization is always worse. However, this section does illustrate that random initialization often produces much worse clustering than k-means counterpart. This is the reason why we provide the particular random seed (`np.random.seed(5)`).

## Takeaway

In this assignment we were able to apply the EM algorithm to a mixture of Gaussians model of text data. This was made possible by modifying the model to assume a diagonal covariance for each cluster, and by modifying the implementation to use a sparse matrix representation. In the second part you explored the role of k-means initialization on the convergence of the model as well as the interpretability of the clusters.